In [41]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasClassifier 
from keras.callbacks import EarlyStopping
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

Wczytanie danych, usunięcie brakujących wartości, nadanie etykiet

In [42]:
train_set = pd.read_csv('adult/adult.data', sep=", ", header=None, engine='python')
test_set = pd.read_csv('adult/adult.test', sep=", ", skiprows=1, header=None, engine='python')

col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation',
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
              'wage_class']

train_set.columns = col_labels
test_set.columns = col_labels

train = train_set.replace('?', np.nan).dropna()
test = test_set.replace('?', np.nan).dropna()

train_set.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Szybki preprocessing

In [43]:
dataset = pd.concat([train, test])

dataset['wage_class'] = dataset.wage_class.replace({'<=50K.': 0, '<=50K': 0, '>50K.': 1, '>50K': 1})

dataset.drop(["fnlwgt"], axis=1, inplace=True)
dataset.drop(["education"], axis=1, inplace=True)

x = dataset.groupby('native_country')["wage_class"].mean()
d = dict(pd.cut(x[x.index != " United-States"], 5, labels=range(5)))
dataset['native_country'] = dataset['native_country'].replace(d)

dataset = pd.get_dummies(dataset, drop_first=True)

train = dataset.iloc[:train.shape[0]]
test = dataset.iloc[train.shape[0]:]

X_train = train.drop("wage_class", axis=1)
y_train = train.wage_class

X_test = test.drop("wage_class", axis=1)
y_test = test.wage_class

/tmp/ipykernel_308463/4187203541.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['wage_class'] = dataset.wage_class.replace({'<=50K.': 0, '<=50K': 0, '>50K.': 1, '>50K': 1})
/tmp/ipykernel_308463/4187203541.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['native_country'] = dataset['native_country'].replace(d)


Standaryzacja

In [44]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

print(f"X_train shape:\t{X_train.shape}")
print(f"X_test shape:\t{X_test.shape}")

X_train shape:	(30162, 41)
X_test shape:	(15060, 41)


In [45]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, shape=[41,]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(shape=shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [46]:
keras_class = KerasClassifier(build_model)
keras_class

KerasClassifier(
	model=<function build_model at 0x7e16ccbca2a0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)

In [47]:
keras_class.fit(X_train, y_train, epochs=100, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)])

Epoch 1/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6637 - loss: 4.0290 - val_accuracy: 0.7812 - val_loss: 2.4860
Epoch 2/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step - accuracy: 0.7887 - loss: 2.1857 - val_accuracy: 0.7988 - val_loss: 1.7792
Epoch 3/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step - accuracy: 0.8011 - loss: 2.0109 - val_accuracy: 0.8177 - val_loss: 1.2994
Epoch 4/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step - accuracy: 0.8063 - loss: 1.4198 - val_accuracy: 0.7451 - val_loss: 4.1083
Epoch 5/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step - accuracy: 0.7537 - loss: 3.9598 - val_accuracy: 0.7451 - val_loss: 4.1045
Epoch 6/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.7487 - loss: 4.0330 - val_accuracy: 0.7451 - val_loss: 4.1086
Epoch 7/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 540us/step - accuracy: 0.7522 - loss: 3.9799 - val_accuracy: 0.7391 - val_loss: 4.1199
Epoch 8/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - accuracy: 0.7471 - loss: 3.9

KerasClassifier(
	model=<function build_model at 0x7e16ccbca2a0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)

# Zad

Wykonajmy RandomizedSearchCV

In [48]:
param_distribs = {
    "model__n_hidden": [0, 1, 2, 3],
    "model__n_neurons": np.arange(1, 100),
}

keras_cls = KerasClassifier(build_model)

rnd_search_cv = RandomizedSearchCV(keras_cls, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/100
566/566 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7541 - loss: 1.5499 - val_accuracy: 0.6161 - val_loss: 5.3768
Epoch 2/100
566/566 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step - accuracy: 0.7141 - loss: 3.9961 - val_accuracy: 0.7345 - val_loss: 4.1999
Epoch 3/100
566/566 ━━━━━━━━━━━━━━━━━━━━ 0s 542us/step - accuracy: 0.7443 - loss: 4.0314 - val_accuracy: 0.7300 - val_loss: 4.3139
Epoch 4/100
566/566 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step - accuracy: 0.7419 - loss: 3.9821 - val_accuracy: 0.7300 - val_loss: 4.1825
Epoch 5/100
566/566 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7604 - loss: 3.7857 - val_accuracy: 0.7404 - val_loss: 4.0383
Epoch 6/100
566/566 ━━━━━━━━━━━━━━━━━━━━ 1s 541us/step - accuracy: 0.7462 - loss: 3.8403 - val_accuracy: 0.7464 - val_loss: 3.9446
Epoch 7/100
566/566 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - accuracy: 0.7769 - loss: 3.4552 - val_accuracy: 0.7688 - val_loss: 3.5160
Epoch 8/100
566/566 ━━━━━━

RandomizedSearchCV(cv=3,
                   estimator=KerasClassifier(model=<function build_model at 0x7e16ccbca2a0>),
                   param_distributions={'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
                   verbose=2)

In [49]:
rnd_search_cv.best_params_

{'model__n_neurons': 73, 'model__n_hidden': 2}

# Zad

Wykonaj RandomizedSearchCV na danych FashionMINT


In [50]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [51]:
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [53]:
X_train = X_train.reshape((X_train.shape[0], -1))
X_valid = X_valid.reshape((X_valid.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

In [67]:
param_distribs = {
    "model__n_hidden": [0, 1, 2, 3],
    "model__n_neurons": np.arange(1, 100),
    "model__shape": [[784]]
}

keras_cls = KerasClassifier(build_model)

rnd_search_cv = RandomizedSearchCV(keras_cls, param_distribs, n_iter=10, cv=2, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=1, mode='min', verbose=1)])

Fitting 2 folds for each of 1 candidates, totalling 2 fits
Epoch 1/100
774/774 ━━━━━━━━━━━━━━━━━━━━ 1s 564us/step - accuracy: 0.0992 - loss: 72.0788 - val_accuracy: 0.0982 - val_loss: 73.2817
Epoch 2/100
774/774 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step - accuracy: 0.1029 - loss: 71.9625 - val_accuracy: 0.0982 - val_loss: 73.2817
Epoch 3/100
774/774 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step - accuracy: 0.1026 - loss: 72.3179 - val_accuracy: 0.0982 - val_loss: 73.2817
Epoch 4/100
774/774 ━━━━━━━━━━━━━━━━━━━━ 0s 463us/step - accuracy: 0.0994 - loss: 73.0273 - val_accuracy: 0.0982 - val_loss: 73.2817
Epoch 5/100
774/774 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - accuracy: 0.1015 - loss: 72.6153 - val_accuracy: 0.0982 - val_loss: 73.2817
Epoch 6/100
774/774 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step - accuracy: 0.0993 - loss: 72.6620 - val_accuracy: 0.0982 - val_loss: 73.2817
Epoch 7/100
774/774 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step - accuracy: 0.0976 - loss: 72.7171 - val_accuracy: 0.0982 - val_loss: 73.2817
Epoch 8/10

KeyboardInterrupt: 

In [ ]:
rnd_search_cv.best_params_